In [ ]:
import json

with open('./data/sessions.json', 'r') as f:
    sessions = json.load(f)

sessions

In [37]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

documents = []
for session in sessions:
    doc = Document(
        page_content = session['abstract'],
        metadata = {
            'title': session['title'],
            'speakers': session['speakers'],
            'room': session['room'],
            'start_time': session['start_time'],
            'end_time': session['end_time']
        }
    )
    documents.append(doc)

embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents, embeddings)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [27]:
from langchain_openai import ChatOpenAI  


llm = ChatOpenAI(  
  model="gpt-4o",  
  temperature=0.7
)

template = """
You are a helpful assistant helping attendees at Nebraska.Code(), 
a software development conference, make the most of their conference 
by answering questions about sessions and helping them find the best 
sessions to attend based on their interests. Your answers should be 
conversational - do not simply respond with the verbatim session data, 
but do make sure to include relevant information such as the room and 
times of the sessions. If there is more than one session that matches 
their query, feel free to reference them all. A bulleted list is 
acceptable in the output.

Here are some sessions that appear relevant to the user's question. 
Use these as you see fit to help answer the user's question:

{context}

Question: {question}
Answer: 

"""

In [ ]:
from langchain import PromptTemplate

prompt = PromptTemplate(
  template=template, 
  input_variables=["context", "question"]
)

In [ ]:
question = "What sessions about unit testing should I go to?"
llm.invoke(question)

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={'k': 10})
docs = retriever.invoke(question)

docs

In [ ]:
p = prompt.invoke({"context": docs, "question": question })
p

In [ ]:
response = llm.invoke(p)
response

In [29]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

chain = (
  {"context": vector_store.as_retriever(search_kwargs={'k': 10}),  "question": RunnablePassthrough()} 
  | prompt 
  | llm
  | StrOutputParser() 
)

In [ ]:
print(chain.invoke(question))